In [27]:
import os
from glob import glob
import numpy as np

from models.VAE import VariationalAutoencoder
from keras.preprocessing.image import ImageDataGenerator


In [28]:
# run params
section = 'vae'
run_id = '0001'
data_name = 'faces'
RUN_FOLDER = 'run/{}/'.format(section)
RUN_FOLDER += '_'.join([run_id, data_name])

if not os.path.exists(RUN_FOLDER):
    os.makedirs(RUN_FOLDER)
    os.makedirs(os.path.join(RUN_FOLDER, 'viz'))
    os.makedirs(os.path.join(RUN_FOLDER, 'images'))
    os.makedirs(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #


DATA_FOLDER = './data/celeb/'

Data

In [29]:
INPUT_DIM = (128,128,3)
BATCH_SIZE = 32

filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))

NUM_IMAGES = len(filenames)
print('DATA_FOLDER: ', DATA_FOLDER)
print (NUM_IMAGES)

DATA_FOLDER:  ./data/celeb/
42


In [30]:
data_gen = ImageDataGenerator(rescale=1./255)

data_flow = data_gen.flow_from_directory(DATA_FOLDER
                                         , target_size = INPUT_DIM[:2]
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         , class_mode = 'input'
                                         , subset = "training"
                                            )

Found 42 images belonging to 1 classes.


In [31]:
vae = VariationalAutoencoder(
                input_dim = INPUT_DIM
                , encoder_conv_filters=[32,64,64, 64]
                , encoder_conv_kernel_size=[3,3,3,3]
                , encoder_conv_strides=[2,2,2,2]
                , decoder_conv_t_filters=[64,64,32,3]
                , decoder_conv_t_kernel_size=[3,3,3,3]
                , decoder_conv_t_strides=[2,2,2,2]
                , z_dim=200
                , use_batch_norm=True
                , use_dropout=True)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [32]:
vae.encoder.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 64, 64, 32)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [33]:
vae.decoder.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              823296    
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_5 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 64)        0   

In [34]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 10000
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [35]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [36]:
vae.train_with_generator(     
    data_flow
    , epochs = EPOCHS
    , steps_per_epoch = NUM_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/200
2/1 [=============================================] - 3s 2s/step - loss: 1218.1305 - vae_r_loss: 1014.3063 - vae_kl_loss: 219.8775

Epoch 00001: saving model to run/vae/0001_faces/weights/weights-001-1234.18.h5

Epoch 00001: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 2/200
2/1 [=============================================] - 0s 21ms/step - loss: 1207.8998 - vae_r_loss: 1000.5715 - vae_kl_loss: 186.0021


/mnt/volume/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.106995). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)



Epoch 00002: saving model to run/vae/0001_faces/weights/weights-002-1186.57.h5

Epoch 00002: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 3/200
2/1 [=============================================] - 0s 29ms/step - loss: 1159.6964 - vae_r_loss: 988.7952 - vae_kl_loss: 175.4749

Epoch 00003: saving model to run/vae/0001_faces/weights/weights-003-1164.27.h5

Epoch 00003: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 4/200
2/1 [=============================================] - 0s 24ms/step - loss: 1162.3981 - vae_r_loss: 980.6282 - vae_kl_loss: 169.8670

Epoch 00004: saving model to run/vae/0001_faces/weights/weights-004-1150.50.h5

Epoch 00004: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 5/200
2/1 [=============================================] - 0s 20ms/step - loss: 1143.4295 - vae_r_loss: 966.7163 - vae_kl_loss: 159.1565

Epoch 00005: saving model to run/vae/0001_faces/weights/weights-005-1125.87.h5

Epoch 00005: saving model to run/vae/0001_f